In [2]:
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from catboost import Pool, cv
random_state = 7
import time
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
import os
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
import warnings
import numpy as np
warnings.filterwarnings("ignore")

In [3]:
# tr = "trainset_ALLvsHealthy_80_target_in"
# ts = "testset_ALLvsHealthy_20_target_in" 

tr = "trainset_AMLvsHealthy_80_target_in"
ts = "testset_AMLvsHealthy_20_target_in" 


# tr = "trainset_AMLvsALL_90_target_in"
# ts = "testset_AMLvsALL_10_target_in"

path = os.getcwd()
parent = os.path.dirname(path)

df_whole_train = pd.read_csv(parent + '\Data\\' + tr + ".csv")
df_whole_test = pd.read_csv(parent + '\Data\\' + ts + ".csv")

In [32]:
import pickle
with open("genes_to_keep.pickle", 'rb') as f:
    keep_pickle = pickle.load(f)

keep = []
for item in keep_pickle:
    keep.append(item)



keep.append("target")

In [33]:
len(keep)

129

In [34]:
df_reduced_train = df_whole_train[keep]
df_reduced_test = df_whole_test[keep]
df_reduced_test

,CCL5,HLA-E,CTTN,RCN1,TGFBI,MYBL2,CD14,ATXN2L,PLK2,NUP214,...,SPX,VPREB1,CDCA8,GDF15,KDM7A,LUZP1,CLIC4P1,SH3BP4,TCL1A,target
0,6.812940,9.585541,6.108900,7.654619,6.557980,7.837997,7.268469,7.153292,4.484727,8.953377,...,7.260695,4.478436,7.521901,5.818731,6.216435,7.329044,6.354620,5.558002,6.797447,1
1,6.810159,9.990029,5.989212,7.123952,9.654628,7.360218,13.104125,7.572187,7.280903,9.206944,...,6.672966,5.695527,6.400406,5.327032,7.958703,7.109189,9.210789,4.702297,6.709682,1
2,7.741660,10.707400,4.635330,7.752940,7.362820,7.226950,7.922950,6.221470,5.568860,9.586280,...,3.338390,4.154330,7.105050,4.865540,6.265700,7.755610,6.911980,4.914900,4.569920,1
3,1432.945653,32362.958365,90.585089,2004.156458,915.898010,953.673703,134.619385,6653.209320,158.962978,4582.548271,...,2.125811,644.969324,467.221735,10.556132,557.316634,1800.696578,0.000000,5.602389,41.458639,1
4,8.257450,10.341300,7.232930,7.797490,6.174470,6.906750,7.508740,5.472260,5.604920,7.875710,...,3.054310,4.602800,6.720270,5.060200,7.278530,8.664670,6.338370,4.611440,4.845510,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1006,7.392190,9.666690,4.494920,8.451970,6.724240,6.638400,7.346210,5.943060,7.096560,8.606580,...,3.341800,3.952560,6.338380,4.569430,8.724820,10.174800,7.702850,4.344970,4.877000,1
1007,5111.269216,42374.882675,15.702881,1027.317069,180.622532,1437.797271,18046.983215,5388.320727,1627.598702,9475.971691,...,2.279853,0.000000,414.702376,16.527268,2418.597521,1577.075173,0.000000,1.756229,59.000231,1
1008,6.449810,10.640900,4.754980,8.248290,5.369920,7.174950,5.726780,6.266230,4.274080,8.430570,...,2.825730,5.488810,6.917320,4.823430,5.393570,6.890750,6.197260,4.010000,4.354390,1
1009,9.186320,10.449400,4.403910,7.327360,5.245890,7.379710,10.785100,5.900560,4.047530,8.512510,...,3.002060,5.130160,6.759990,4.383920,6.750430,5.849690,7.566420,4.837310,5.139460,1


In [35]:
df_reduced_train.to_csv(parent + '\Data\\' + "trainset_AML_vs_Healthy_200.csv", index=False)
df_reduced_test.to_csv(parent + '\Data\\' + "testset_AML_vs_Healthy_200.csv", index=False)

# AML vs Healthy not in literature

In [3]:
# tr = "trainset_ALLvsHealthy_80_target_in"
# ts = "testset_ALLvsHealthy_20_target_in" 

tr = "trainset_AML_vs_Healthy_100"
ts = "testset_AML_vs_Healthy_100"


# tr = "trainset_AMLvsALL_90_target_in"
# ts = "testset_AMLvsALL_10_target_in"

path = os.getcwd()
parent = os.path.dirname(path)

df_whole_train = pd.read_csv(parent + '\Data\\' + tr + ".csv")
df_whole_test = pd.read_csv(parent + '\Data\\' + ts + ".csv")

In [7]:
df_train_not_literature = df_whole_train.drop(["HLA-E", "TGFBI", "MYBL2", "CD14", "NUP214", "EPS8", "IGF1R", "PRAME", "P2RX4", "HOXB6", "WT1", "FLT3", "CDKN2A", "GPR65", "HOXA1", "LPAR6", "SMYD3", "DEPTOR", "VPREB1", "GDF15", "TCL1A", "COMMD3-BMI1"], axis=1)
df_test_not_literature = df_whole_test.drop(["HLA-E", "TGFBI", "MYBL2", "CD14", "NUP214", "EPS8", "IGF1R", "PRAME", "P2RX4", "HOXB6", "WT1", "FLT3", "CDKN2A", "GPR65", "HOXA1", "LPAR6", "SMYD3", "DEPTOR", "VPREB1", "GDF15", "TCL1A", "COMMD3-BMI1"], axis=1)

In [9]:
df_train_not_literature.to_csv(parent + '\Data\\' + "trainset_AML_vs_Healthy_100_not_in_literature.csv", index=False)
df_test_not_literature.to_csv(parent + '\Data\\' + "testset_AML_vs_Healthy_100_not_in_literature.csv", index=False)

# ALL vs Healthy not in literature

In [10]:
# tr = "trainset_ALLvsHealthy_80_target_in"
# ts = "testset_ALLvsHealthy_20_target_in" 

tr = "trainset_ALL_vs_Healthy_150"
ts = "testset_ALL_vs_Healthy_150"


# tr = "trainset_AMLvsALL_90_target_in"
# ts = "testset_AMLvsALL_10_target_in"

path = os.getcwd()
parent = os.path.dirname(path)

df_whole_train = pd.read_csv(parent + '\Data\\' + tr + ".csv")
df_whole_test = pd.read_csv(parent + '\Data\\' + ts + ".csv")

In [11]:
df_train_not_literature = df_whole_train.drop(["STMN1" , "SOX4" ,"TGFBI" ,"AEBP1" ,"TSPAN7" ,"S100A13" ,"LYN" ,"CD93" ,"SIRPA" ,"SSBP2" ,"MAP3K5" ,"MPO" ,"P2RX4" ,"BCHE" ,"CTSG" ,"ADAM28" ,"NPY" ,"RHAG" ,"ARG1" ,"CEACAM8" ,"CCR2" ,"MMP8" ,"PSMA6" ,"THY1" ,"SLC1A5" ,"SPP1" ,"DNTT" ,"IGHV3-23" ,"ZNF423" ,"ADAM10" ,"ADA" ,"LPAR6" ,"CD244" ,"ARPP21"], axis=1)
df_test_not_literature = df_whole_test.drop(["STMN1" , "SOX4" ,"TGFBI" ,"AEBP1" ,"TSPAN7" ,"S100A13" ,"LYN" ,"CD93" ,"SIRPA" ,"SSBP2" ,"MAP3K5" ,"MPO" ,"P2RX4" ,"BCHE" ,"CTSG" ,"ADAM28" ,"NPY" ,"RHAG" ,"ARG1" ,"CEACAM8" ,"CCR2" ,"MMP8" ,"PSMA6" ,"THY1" ,"SLC1A5" ,"SPP1" ,"DNTT" ,"IGHV3-23" ,"ZNF423" ,"ADAM10" ,"ADA" ,"LPAR6" ,"CD244" ,"ARPP21"], axis=1)

In [12]:
df_train_not_literature.to_csv(parent + '\Data\\' + "trainset_ALL_vs_Healthy_150_not_in_literature.csv", index=False)
df_test_not_literature.to_csv(parent + '\Data\\' + "testset_ALL_vs_Healthy_150_not_in_literature.csv", index=False)

In [13]:
df_train_not_literature

,WARS,SDCBP,CST3,COL3A1,RER1,MATN2,IFI27,DBN1,C1QB,SMARCD1,...,MED28,TMEM106B,DPPA4,L2HGDH,VRTN,FOXRED2,ASTE1,SEMA4C,PRR5,target
0,7.592169,8.838503,6.151441,5.382015,6.986188,4.224580,6.065428,7.298434,5.819314,7.216487,...,6.744962,4.534215,4.482342,5.210566,7.992622,7.182100,6.811301,7.154131,7.747942,1
1,7.155810,8.598000,5.811540,5.570950,5.995070,3.248400,5.778960,7.749000,4.179420,7.505220,...,5.194250,6.054520,3.779860,5.581250,7.293750,7.037100,5.630670,8.184840,6.182200,1
2,8.024406,9.603805,7.271123,5.090965,7.605913,4.121849,6.723201,8.322849,5.595167,7.531628,...,8.045335,5.530472,4.431018,5.140376,6.653190,7.246701,6.717686,6.786491,7.593878,1
3,8.384836,8.025827,5.443172,5.663897,7.235209,4.660694,8.245308,7.919013,6.424624,6.638817,...,8.128368,4.525453,4.700825,4.878005,7.846647,5.792978,7.379145,6.545153,7.576077,1
4,10.425700,11.747700,11.395600,4.267280,8.705490,3.450720,4.897780,6.669080,4.627040,7.340240,...,6.068870,8.312920,3.308290,4.947730,5.746580,6.158910,6.388380,7.674050,7.392490,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3493,8.482300,9.995340,5.854950,4.348700,9.123270,3.120460,4.635130,8.183680,4.243090,8.120660,...,7.790490,7.861220,3.081050,5.650910,5.979150,6.903630,6.572240,6.233320,6.300700,1
3494,8.622109,9.834865,8.677433,5.207763,7.678730,4.261137,5.861557,7.474985,5.491992,6.807649,...,7.969268,5.365993,6.118198,5.042093,6.288468,6.665375,7.448243,7.820779,8.212433,0
3495,8.196201,9.926119,7.577701,6.877518,8.485634,3.961185,8.732912,8.759654,7.129287,6.564231,...,8.889535,6.361668,4.364104,4.706063,6.767586,5.965524,7.142933,6.580128,6.728891,1
3496,8.392720,8.161030,7.076030,4.089110,7.220750,3.277080,5.446460,9.366650,4.337540,8.123340,...,5.256730,6.853560,3.572970,5.408070,6.313730,6.857840,6.575740,6.089020,6.452180,1
